
# Human Reinforcement Learning Model

This notebook implements a human reinforcement learning model that:
1. Loads pre-trained weights from the previous file.
2. Prompts the user to input characters `0-9`, `a-z`, and `A-Z`.
3. Requests the user to provide input again, prioritizing characters in the order of lowest to highest accuracy, as determined by the earlier model.


In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from collections import defaultdict
import tkinter as tk
from PIL import ImageGrab, Image
import csv


In [2]:
# Enhanced CNN
class EnhancedCNNModel(nn.Module):
    def __init__(self, num_classes):
        super(EnhancedCNNModel, self).__init__()

        self.num_classes = num_classes

        # Convolutional blocks
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # Output: 32x320x240
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 32x160x120

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Output: 64x160x120
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 64x80x60

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # Output: 128x80x60
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 128x40x30

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)  # Output: 256x40x30
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 256x20x15

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)  # Output: 512x20x15
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 512x10x7

        # Fully connected layers are initialized dynamically
        self.fc1 = None
        self.fc2 = None
        self.fc3 = None

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def initialize_fc_layers(self, input_shape):
        """Initialize the fully connected layers dynamically based on the input shape."""
        with torch.no_grad():
            dummy_input = torch.zeros(1, *input_shape)
            x = self.pool1(self.bn1(self.conv1(dummy_input)))
            x = self.pool2(self.conv2(x))
            x = self.pool3(self.conv3(x))
            x = self.pool4(self.conv4(x))
            x = self.pool5(self.conv5(x))
            flattened_size = x.numel()  # Total elements after flattening

        # Dynamically initialize fully connected layers
        self.fc1 = nn.Linear(flattened_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, self.num_classes)

        # Print the initialized sizes for debugging
        print(f"Initialized fc1 with input size {flattened_size} and output size 512")
        print(f"Initialized fc2 with input size 512 and output size 256")
        print(f"Initialized fc3 with input size 256 and output size {self.num_classes}")

    def forward(self, x):
        # Convolutional blocks
        x = self.pool1(self.bn1(self.conv1(x)))
        x = self.pool2(self.conv2(x))
        x = self.pool3(self.conv3(x))
        x = self.pool4(self.conv4(x))
        x = self.pool5(self.conv5(x))

        # Flatten and pass through fully connected layers
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)

        return x

In [ ]:
# Instantiate the model and load the weights
num_classes = 62  # 10 digits + 26 lowercase + 26 uppercase
model = EnhancedCNNModel(num_classes=num_classes)

# Initialize the fully connected layers
input_shape = (1, 160, 120)  # Example input shape, adjust as necessary
model.initialize_fc_layers(input_shape)

weights_path = './weights/79accuracy.pth'
model.load_state_dict(torch.load(weights_path, map_location=torch.device('cpu')))
model.eval()

print("Model loaded successfully!")

Initialized fc1 with input size 7680 and output size 512
Initialized fc2 with input size 512 and output size 256
Initialized fc3 with input size 256 and output size 62
Model loaded successfully!


C:\Users\natha\AppData\Local\Temp\ipykernel_41576\2839057638.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weights_path, map_location

In [ ]:
# Global variable to store the last window position
last_window_position = {"x": 750, "y": 750}  # Default initial position


def draw_letter_prompt(letter, save_folder="user_drawings"):
    """Create a drawing interface for the user to draw a given letter and save the image."""
    
    exit = False
    
    global last_window_position

    # Ensure the save folder exists
    os.makedirs(save_folder, exist_ok=True)

    # Create the main application window
    root = tk.Tk()
    root.title(f"Draw the Letter: {letter}")

    # Use the last recorded window position
    window_x, window_y = last_window_position["x"], last_window_position["y"]
    root.geometry(f"+{window_x}+{window_y}")

    # Canvas for drawing
    canvas_width, canvas_height = 320, 240
    canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg="white")
    canvas.pack()

    # Variables to store drawing state
    drawing = False
    last_x, last_y = None, None
    drawn_image = None  # To store the drawn image

    def start_draw(event):
        """Start drawing."""
        nonlocal drawing, last_x, last_y
        drawing = True
        last_x, last_y = event.x, event.y

    def draw(event):
        """Draw lines on the canvas."""
        nonlocal drawing, last_x, last_y
        if drawing:
            canvas.create_line(last_x, last_y, event.x, event.y, width=5, fill="black")
            last_x, last_y = event.x, event.y

    def stop_draw(event):
        """Stop drawing."""
        nonlocal drawing
        drawing = False

    def clear_canvas():
        """Clear the canvas."""
        canvas.delete("all")

    def save_drawing(file_name=None):
        """Save the drawn image."""
        nonlocal drawn_image
        x = root.winfo_rootx() + canvas.winfo_x()
        y = root.winfo_rooty() + canvas.winfo_y()
        x1 = x + canvas.winfo_width()
        y1 = y + canvas.winfo_height()
        # Capture the canvas area as an image
        image = ImageGrab.grab((x, y, x1, y1)).convert("L").resize((160, 120))
        # Convert image to a NumPy array (normalized)
        drawn_image = np.array(image) / 255.0
        # Determine the image file name
        image_file_name = f"{file_name if file_name else letter}.png"
        # Save the image to the specified folder
        image_path = os.path.join(save_folder, image_file_name)
        ImageGrab.grab((x, y, x1, y1)).convert("L").save(image_path)
        print(f"Saved image for letter '{letter}' at {image_path}")
        root.destroy()  # Close the window

    def update_window_position():
        """Update the last window position when the window moves."""
        global last_window_position
        last_window_position["x"] = root.winfo_x()
        last_window_position["y"] = root.winfo_y()

    def on_closing():
        """Handle window close event."""
        print("Window closed by user. Exiting program.")
        update_window_position()
        exit = True
        root.destroy()

    # Bind mouse events
    canvas.bind("<ButtonPress-1>", start_draw)
    canvas.bind("<B1-Motion>", draw)
    canvas.bind("<ButtonRelease-1>", stop_draw)

    # Bind window close event
    root.protocol("WM_DELETE_WINDOW", on_closing)

    # Bind window movement event
    root.bind("<Configure>", lambda e: update_window_position())

    # Add buttons
    button_frame = tk.Frame(root)
    button_frame.pack()

    clear_button = tk.Button(button_frame, text="Clear", command=clear_canvas)
    clear_button.pack(side="left", padx=10)

    submit_button = tk.Button(button_frame, text="Submit", command=save_drawing)
    submit_button.pack(side="left", padx=10)

    # Run the Tkinter main loop
    root.mainloop()

    # Ensure the drawn image is returned
    return None if exit else drawn_image

In [5]:
def collect_or_load_images(characters, save_folder="user_drawings"):
    """
    Collect user input for characters if not saved, otherwise load existing images.
    """
    # Ensure the save folder exists
    os.makedirs(save_folder, exist_ok=True)

    # Path to the CSV file
    csv_file_path = os.path.join(save_folder, "image_mapping.csv")
    image_mapping = defaultdict(list)

    # Load existing mappings from CSV if available
    if os.path.exists(csv_file_path):
        with open(csv_file_path, mode='r') as csv_file:
            csv_reader = csv.reader(csv_file)
            next(csv_reader)  # Skip header
            for row in csv_reader:
                image_path, char = row
                full_image_path = os.path.join(save_folder, image_path)
                if os.path.exists(full_image_path):
                    # Load image and add to mapping
                    image = Image.open(full_image_path).convert("L")
                    image = image.resize((160, 120))  # Resize to match input size
                    image_array = np.array(image) / 255.0  # Normalize
                    image_mapping[char].append((full_image_path, image_array))
                else:
                    print(f"Warning: Missing file {full_image_path} listed in CSV.")

    # Check if all characters are available
    for char in characters:
        if char not in image_mapping or len(image_mapping[char]) == 0:
            print(f"Character '{char}' is missing. Please draw it.")
            user_input = draw_letter_prompt(char, save_folder=save_folder)
            if user_input is not None:
                # Generate sequential filename
                next_image_id = len(image_mapping) + 1
                image_file_name = f"{next_image_id:04d}.png"
                image_path = os.path.join(save_folder, image_file_name)

                # Save the drawn image
                img_pil = Image.fromarray((user_input * 255).astype(np.uint8))
                img_pil.save(image_path)

                # Update the mapping
                image_mapping[char].append((image_path, user_input))

                # Append to CSV
                with open(csv_file_path, mode='a', newline='') as csv_file:
                    csv_writer = csv.writer(csv_file)
                    csv_writer.writerow([os.path.relpath(image_path, start=save_folder), char])

    return image_mapping


def load_images_from_csv(save_folder="user_drawings"):
    """
    Load images and labels from the CSV file.
    """
    csv_file_path = os.path.join(save_folder, "image_mapping.csv")
    images = []
    labels = []

    if not os.path.exists(csv_file_path):
        print(f"No CSV mapping file found in {save_folder}.")
        return images, labels

    with open(csv_file_path, mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)  # Skip header
        for row in csv_reader:
            image_path, label = row
            full_image_path = os.path.join(save_folder, image_path)
            if os.path.exists(full_image_path):
                # Load the image
                image = Image.open(full_image_path).convert("L")
                image = image.resize((160, 120))  # Resize to match input size
                image_array = np.array(image) / 255.0  # Normalize
                images.append(image_array)
                labels.append(label)
            else:
                print(f"Missing file {full_image_path} listed in CSV.")

    return images, labels


In [ ]:
# Reinforcement loop
def reinforcement_loop(model, characters, user_images, save_folder="user_drawings"):
    while True:
        y_true = []
        y_pred = []

        # Predict user inputs and collect true/predicted labels
        for char in characters:
            for img in user_images[char]:
                img_tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
                with torch.no_grad():
                    output = model(img_tensor)
                    _, predicted_idx = torch.max(output, 1)
                y_true.append(char)
                y_pred.append(characters[predicted_idx])

        # Compute confusion matrix
        cm = confusion_matrix(y_true, y_pred, labels=characters)

        # Accuracy per character
        accuracy_per_char = np.diag(cm) / np.sum(cm, axis=1, where=np.sum(cm, axis=1) != 0)
        char_accuracy_dict = dict(zip(characters, accuracy_per_char))

        # Display current accuracy
        print("\nCurrent Accuracy Per Character:")
        for char, accuracy in char_accuracy_dict.items():
            print(f"{char}: {accuracy:.2%}")

        # Filter characters below 85% accuracy
        below_85_accuracy = [char for char, acc in char_accuracy_dict.items() if acc < 0.85]

        # Break the loop if all characters are more than 85% accurate
        if not below_85_accuracy:
            print("\nAll characters have achieved more than 85% accuracy. Process complete!")
            break

        # Re-prompt user for characters below 85% accuracy
        print("\nRe-prompting for characters below 85% accuracy:")
        for char in below_85_accuracy:
            print(f"Draw the character: {char}")
            user_input = draw_letter_prompt(char, save_folder=save_folder)
            if user_input is None:
                break
            user_images[char].append(user_input)

            # Save the new input to the CSV
            csv_file_path = os.path.join(save_folder, "image_mapping.csv")
            next_image_id = sum(len(imgs) for imgs in user_images.values()) + 1
            image_file_name = f"{next_image_id:04d}.png"
            image_path = os.path.join(save_folder, image_file_name)
            
            # Save the image
            img_pil = Image.fromarray((user_input * 255).astype(np.uint8))
            img_pil.save(image_path)

            # Append to CSV
            with open(csv_file_path, mode='a', newline='') as csv_file:
                csv_writer = csv.writer(csv_file)
                csv_writer.writerow([os.path.relpath(image_path, start=save_folder), char])


In [7]:
characters = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
save_folder = "user_drawings"
user_images = collect_or_load_images(characters, save_folder=save_folder)
reinforcement_loop(model, characters, user_images, save_folder=save_folder)

Character '0' is missing. Please draw it.
Saved image for letter '0' at user_drawings\0.png
Character '1' is missing. Please draw it.
Saved image for letter '1' at user_drawings\1.png
Character '2' is missing. Please draw it.
Saved image for letter '2' at user_drawings\2.png
Character '3' is missing. Please draw it.
Saved image for letter '3' at user_drawings\3.png
Character '4' is missing. Please draw it.
Saved image for letter '4' at user_drawings\4.png
Character '5' is missing. Please draw it.
Saved image for letter '5' at user_drawings\5.png
Character '6' is missing. Please draw it.
Saved image for letter '6' at user_drawings\6.png
Character '7' is missing. Please draw it.
Saved image for letter '7' at user_drawings\7.png
Character '8' is missing. Please draw it.
Saved image for letter '8' at user_drawings\8.png
Character '9' is missing. Please draw it.
Saved image for letter '9' at user_drawings\9.png
Character 'A' is missing. Please draw it.
Saved image for letter 'A' at user_dra

ValueError: too many dimensions 'str'